In [1]:
# --- CELL 1: FINE-TUNING SETUP ---
import tensorflow as tf
from tensorflow.keras import layers, models, applications, callbacks, optimizers, regularizers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
FINE_TUNE_EPOCHS = 20
PATIENCE = 5
LEARNING_RATE_FT = 1e-5  # Lower LR for fine-tuning
FIG_DIR = "figures_finetune"
MODEL_DIR = "models_finetune"

os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# Load best pre-trained models
nasnet_model = tf.keras.models.load_model("models/NASNetMobile_best.keras")
inception_model = tf.keras.models.load_model("models/InceptionV3_best.keras")

In [2]:
# --- CELL 2: NASNETMOBILE FINE-TUNING (FIXED) ---

def unfreeze_and_fine_tune_nasnet(model, unfreeze_layers=20):
    """Unfreeze last layers of NASNetMobile with NAS-specific regularization"""
    
    # Find the NASNetMobile base model layer
    # It's typically the 4th layer in the functional API model
    base_model = None
    for layer in model.layers:
        if 'nasnet' in layer.name.lower():
            base_model = layer
            break
    
    if base_model is None:
        print("NASNetMobile base model not found. Using layer 4 as fallback.")
        base_model = model.layers[4]
    
    # Unfreeze the base model
    base_model.trainable = True
    
    # Freeze all but the last N layers of the base model
    for layer in base_model.layers[:-unfreeze_layers]:
        layer.trainable = False
        # Add regularization to unfrozen layers
    for layer in base_model.layers[-unfreeze_layers:]:
        if hasattr(layer, 'kernel_regularizer'):
            layer.kernel_regularizer = regularizers.l2(0.0001)
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=optimizers.Adam(
            learning_rate=LEARNING_RATE_FT,
            clipnorm=1.0
        ),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'auc']
    )
    
    print(f"Unfroze last {unfreeze_layers} layers of {base_model.name}")
    print(f"Trainable layers: {sum([l.trainable for l in model.layers])}/{len(model.layers)}")
    
    return model

# Apply fine-tuning to NASNetMobile
nasnet_ft = unfreeze_and_fine_tune_nasnet(nasnet_model)

# NASNetMobile-specific callbacks
nasnet_callbacks = [
    callbacks.EarlyStopping(patience=PATIENCE, restore_best_weights=True, min_delta=0.0001),
    callbacks.ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-7),
    callbacks.ModelCheckpoint(f"{MODEL_DIR}/nasnet_ft_best.keras", save_best_only=True)
]

Unfroze last 20 layers of nasnet_mobile
Trainable layers: 6/6


In [3]:
# --- CELL 3: INCEPTIONV3 FINE-TUNING (FIXED) ---

def unfreeze_and_fine_tune_inception(model, unfreeze_layers=15):
    """Unfreeze InceptionV3 with precision-focused regularization"""
    
    # Find the InceptionV3 base model layer
    base_model = None
    for layer in model.layers:
        if 'inception' in layer.name.lower() and 'v3' in layer.name.lower():
            base_model = layer
            break
    
    if base_model is None:
        print("InceptionV3 base model not found. Using layer 4 as fallback.")
        base_model = model.layers[4]
    
    # Unfreeze the base model
    base_model.trainable = True
    
    # Freeze all but the last N layers
    for layer in base_model.layers[:-unfreeze_layers]:
        layer.trainable = False
    
    # Add regularization to unfrozen layers
    for layer in base_model.layers[-unfreeze_layers:]:
        if hasattr(layer, 'kernel_regularizer'):
            layer.kernel_regularizer = regularizers.l2(0.0005)
    
    # Recompile with precision focus
    model.compile(
        optimizer=optimizers.Adam(
            learning_rate=LEARNING_RATE_FT/2,
            clipvalue=0.5
        ),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall', 'auc']
    )
    
    print(f"Unfroze last {unfreeze_layers} layers of {base_model.name}")
    print(f"Trainable layers: {sum([l.trainable for l in model.layers])}/{len(model.layers)}")
    
    return model

# Apply fine-tuning to InceptionV3
inception_ft = unfreeze_and_fine_tune_inception(inception_model)

# Precision-focused callbacks
inception_callbacks = [
    callbacks.EarlyStopping(monitor='val_precision', patience=PATIENCE, mode='max', restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_precision', factor=0.3, patience=2, mode='max'),
    callbacks.ModelCheckpoint(f"{MODEL_DIR}/inception_ft_best.keras", monitor='val_precision', save_best_only=True, mode='max')
]

Unfroze last 15 layers of inception_v3
Trainable layers: 6/6


In [5]:
# --- LOAD DATASETS ---
def prepare_dataset(dir_path, shuffle=True):
    ds = tf.keras.preprocessing.image_dataset_from_directory(
        dir_path,
        labels="inferred", 
        label_mode="binary",
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        shuffle=shuffle
    )
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32), y))  # No /255.0 - preprocessing handled by model
    return ds

# Load the datasets
train_ds = prepare_dataset("train", shuffle=True)
val_ds = prepare_dataset("val", shuffle=False) 
test_ds = prepare_dataset("test", shuffle=False)

# Optimize the pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("Datasets loaded successfully!")
print(f"Train batches: {len(train_ds)}, Val batches: {len(val_ds)}, Test batches: {len(test_ds)}")

Found 1120 files belonging to 2 classes.
Found 229 files belonging to 2 classes.
Found 237 files belonging to 2 classes.
Datasets loaded successfully!
Train batches: 35, Val batches: 8, Test batches: 8


In [6]:
# --- CELL 4: RUN FINE-TUNING FOR INCEPTIONV3 ---
print("Starting InceptionV3 fine-tuning...")

history = inception_ft.fit(
    train_ds,
    validation_data=val_ds,
    epochs=FINE_TUNE_EPOCHS,
    callbacks=inception_callbacks,
    verbose=2
)

# Evaluate fine-tuned model
print("\nEvaluating fine-tuned InceptionV3...")
test_results = inception_ft.evaluate(test_ds, verbose=0)
print(f"Fine-tuned Test Accuracy: {test_results[1]:.4f}")
print(f"Fine-tuned Test Precision: {test_results[2]:.4f}")
print(f"Fine-tuned Test AUC: {test_results[4]:.4f}")

# Save final model
inception_ft.save(f"{MODEL_DIR}/inceptionv3_final.keras")
print("Fine-tuned model saved!")

Starting InceptionV3 fine-tuning...
Epoch 1/20
35/35 - 44s - 1s/step - accuracy: 0.9018 - auc: 0.9853 - loss: 0.2462 - precision: 0.8486 - recall: 0.9843 - val_accuracy: 0.9563 - val_auc: 0.9918 - val_loss: 0.1307 - val_precision: 0.9817 - val_recall: 0.9304 - learning_rate: 5.0000e-06
Epoch 2/20
35/35 - 29s - 834ms/step - accuracy: 0.9366 - auc: 0.9904 - loss: 0.1761 - precision: 0.9026 - recall: 0.9826 - val_accuracy: 0.9563 - val_auc: 0.9917 - val_loss: 0.1333 - val_precision: 0.9817 - val_recall: 0.9304 - learning_rate: 5.0000e-06
Epoch 3/20
35/35 - 31s - 888ms/step - accuracy: 0.9429 - auc: 0.9892 - loss: 0.1650 - precision: 0.9223 - recall: 0.9704 - val_accuracy: 0.9563 - val_auc: 0.9922 - val_loss: 0.1359 - val_precision: 0.9817 - val_recall: 0.9304 - learning_rate: 5.0000e-06
Epoch 4/20
35/35 - 29s - 826ms/step - accuracy: 0.9411 - auc: 0.9863 - loss: 0.1702 - precision: 0.9380 - recall: 0.9478 - val_accuracy: 0.9563 - val_auc: 0.9919 - val_loss: 0.1358 - val_precision: 0.9817 

In [7]:
# --- CELL 4: RUN NASNETMOBILE FINE-TUNING ---
print("Starting NASNetMobile fine-tuning...")

history_nasnet = nasnet_ft.fit(
    train_ds,
    validation_data=val_ds,
    epochs=FINE_TUNE_EPOCHS,
    callbacks=nasnet_callbacks,
    verbose=2
)

# Evaluate fine-tuned model
print("\nEvaluating fine-tuned NASNetMobile...")
test_results_nasnet = nasnet_ft.evaluate(test_ds, verbose=0)
print(f"Fine-tuned Test Accuracy: {test_results_nasnet[1]:.4f}")
print(f"Fine-tuned Test Precision: {test_results_nasnet[2]:.4f}") 
print(f"Fine-tuned Test AUC: {test_results_nasnet[4]:.4f}")

# Save final model
nasnet_ft.save(f"{MODEL_DIR}/nasnet_final.keras")
print("Fine-tuned NASNetMobile saved!")

Starting NASNetMobile fine-tuning...
Epoch 1/20
35/35 - 58s - 2s/step - accuracy: 0.9536 - auc: 0.9875 - loss: 0.1346 - precision: 0.9548 - recall: 0.9548 - val_accuracy: 0.9432 - val_auc: 0.9939 - val_loss: 0.1159 - val_precision: 0.9722 - val_recall: 0.9130 - learning_rate: 1.0000e-05
Epoch 2/20
35/35 - 27s - 775ms/step - accuracy: 0.9473 - auc: 0.9880 - loss: 0.1363 - precision: 0.9510 - recall: 0.9461 - val_accuracy: 0.9432 - val_auc: 0.9941 - val_loss: 0.1168 - val_precision: 0.9722 - val_recall: 0.9130 - learning_rate: 1.0000e-05
Epoch 3/20
35/35 - 28s - 808ms/step - accuracy: 0.9545 - auc: 0.9914 - loss: 0.1195 - precision: 0.9613 - recall: 0.9496 - val_accuracy: 0.9432 - val_auc: 0.9941 - val_loss: 0.1171 - val_precision: 0.9722 - val_recall: 0.9130 - learning_rate: 1.0000e-05
Epoch 4/20
35/35 - 27s - 783ms/step - accuracy: 0.9545 - auc: 0.9884 - loss: 0.1286 - precision: 0.9645 - recall: 0.9461 - val_accuracy: 0.9476 - val_auc: 0.9941 - val_loss: 0.1178 - val_precision: 0.9813

In [8]:
# --- CELL 5: CREATE ENSEMBLE ---
def create_weighted_ensemble(model1, model2, test_ds, weight1=0.6, weight2=0.4):
    """Create weighted ensemble of two models"""
    print("Generating ensemble predictions...")
    
    # Get predictions from both models
    preds1 = model1.predict(test_ds, verbose=0)
    preds2 = model2.predict(test_ds, verbose=0)
    
    # Weighted average (NASNetMobile gets higher weight since it performed better)
    ensemble_pred_probs = (preds1 * weight1) + (preds2 * weight2)
    ensemble_pred = (ensemble_pred_probs > 0.5).astype(int).flatten()
    
    return ensemble_pred, ensemble_pred_probs

# Create ensemble (60% NASNetMobile, 40% InceptionV3)
ensemble_pred, ensemble_probs = create_weighted_ensemble(nasnet_ft, inception_ft, test_ds, weight1=0.6, weight2=0.4)

# Get true labels
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# Evaluate ensemble performance
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report

ensemble_acc = accuracy_score(y_true, ensemble_pred)
ensemble_precision = precision_score(y_true, ensemble_pred)
ensemble_recall = recall_score(y_true, ensemble_pred)
ensemble_auc = roc_auc_score(y_true, ensemble_probs)

print(f"\n🎯 ENSEMBLE PERFORMANCE (NASNetMobile 60% + InceptionV3 40%):")
print(f"Accuracy:   {ensemble_acc:.4f}")
print(f"Precision:  {ensemble_precision:.4f}")
print(f"Recall:     {ensemble_recall:.4f}")
print(f"AUC:        {ensemble_auc:.4f}")

# Compare with individual models
print(f"\n📊 COMPARISON:")
print(f"               Accuracy  Precision  AUC")
print(f"NASNetMobile:   {test_results_nasnet[1]:.4f}    {test_results_nasnet[2]:.4f}     {test_results_nasnet[4]:.4f}")
print(f"InceptionV3:    {test_results[1]:.4f}    {test_results[2]:.4f}     {test_results[4]:.4f}")
print(f"Ensemble:       {ensemble_acc:.4f}    {ensemble_precision:.4f}     {ensemble_auc:.4f}")

# Confusion Matrix
print(f"\n📈 CONFUSION MATRIX:")
print(confusion_matrix(y_true, ensemble_pred))

# Classification Report
print(f"\n📝 CLASSIFICATION REPORT:")
print(classification_report(y_true, ensemble_pred))

# Save ensemble predictions for analysis
ensemble_results = pd.DataFrame({
    'true_label': y_true,
    'nasnet_probs': nasnet_ft.predict(test_ds, verbose=0).flatten(),
    'inception_probs': inception_ft.predict(test_ds, verbose=0).flatten(),
    'ensemble_probs': ensemble_probs.flatten(),
    'ensemble_pred': ensemble_pred
})
ensemble_results.to_csv(f"{LOG_DIR}/ensemble_predictions.csv", index=False)
print(f"\n💾 Ensemble predictions saved to: {LOG_DIR}/ensemble_predictions.csv")

Generating ensemble predictions...

🎯 ENSEMBLE PERFORMANCE (NASNetMobile 60% + InceptionV3 40%):
Accuracy:   0.9747
Precision:  1.0000
Recall:     0.9478
AUC:        0.9872

📊 COMPARISON:
               Accuracy  Precision  AUC
NASNetMobile:   0.9705    0.9909     0.9875
InceptionV3:    0.9620    0.9907     0.9815
Ensemble:       0.9747    1.0000     0.9872

📈 CONFUSION MATRIX:
[[122   0]
 [  6 109]]

📝 CLASSIFICATION REPORT:
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98       122
         1.0       1.00      0.95      0.97       115

    accuracy                           0.97       237
   macro avg       0.98      0.97      0.97       237
weighted avg       0.98      0.97      0.97       237



ValueError: Per-column arrays must each be 1-dimensional